In [19]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


In [20]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [ ]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

balanceamento dos dados

In [21]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')
train_oversampling= pd.read_csv('train_oversampling.csv')

y_test= test['classe']
y_train= train['classe']
y_train_oversampling= train_oversampling['classe']
y_val=val['classe']
classes = test['classe'].unique()

In [22]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())
print("\nconjunto de teste: ", y_train_oversampling.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64

conjunto de teste:  classe
-1    2375
 0    2375
 1    2375
Name: count, dtype: int64


In [ ]:
'''from sklearn.utils import resample

# Separar cada classe
class_min = train[train['classe'] == -1]
class_med = train[train['classe'] == 1]
class_max = train[train['classe'] == 0]

# Número da classe maior
n_samples = len(class_max)

# Replicar as classes menores até o tamanho da maior
class_min_upsampled = resample(class_min, 
                            replace=True, 
                            n_samples=n_samples, 
                            random_state=42)

class_med_upsampled = resample(class_med, 
                            replace=True, 
                            n_samples=n_samples, 
                          random_state=42)

train_oversampling = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

# baralhar o conjunto de treino
train_oversampling = train_oversampling.sample(frac=1, random_state=42).reset_index(drop=True)

print(train_oversampling['classe'].value_counts())

train_oversampling.to_csv('train_oversampling.csv', index=False)'''


classe
 1    2375
 0    2375
-1    2375
Name: count, dtype: int64


tokeinização

In [23]:
stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
train_oversampling['tokens'] = train_oversampling['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
train_oversampling['tokens'] = train_oversampling['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
train_oversampling['stems'] = train_oversampling['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
print(train['stems'].head())

y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()

0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Construção dos modelos de representação

## TF-IDF - fred

In [24]:
def TF_IDF(train, train_oversampling, val, test):
    tf_vect = TfidfVectorizer(ngram_range =(1, 2), min_df =3, max_df =0.5, max_features =500)

        #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_train_oversampling = train_oversampling['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tf_vect.fit_transform(text_train)
    X_train_oversampling = tf_vect.fit_transform(text_train_oversampling)
    X_val   = tf_vect.transform(text_val)
    X_test  = tf_vect.transform(text_test)
    return X_train, X_train_oversampling, X_val, X_test

In [25]:
X_train_tfidf_vec,X_train_oversampling_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = TF_IDF(train,train_oversampling, val, test)

## CountVectorizer -didi

In [40]:
import optuna
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective(trial):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3)  # garante ngram_max >= ngram_min

    # CountVectorizer com parâmetros sugeridos
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(ngram_min, ngram_max),
        strip_accents='unicode',
        max_features=max_features
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)

    y_train = train['classe'] 
    y_val   = val['classe']

    # Treinar Logistic Regression
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    # Métrica: F1 macro
    score = f1_score(y_val, preds, average='macro')
    return score 

def count_vec(train, val, test, best_params):
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        strip_accents='unicode',
        max_features=best_params['max_features']
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    # Fit no treino original, transform nos outros conjuntos
    X_train = c_vect.fit_transform(text_train)
    X_val = c_vect.transform(text_val)
    X_test = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [49]:
study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=30) 
print("Melhores parâmetros:", study.best_params)
print("Melhor F1:", study.best_value)
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train, val, test, study.best_params)

[I 2025-11-11 16:23:21,980] A new study created in memory with name: no-name-80f90efb-00dc-4f64-b5cc-8f9c119539c8


[I 2025-11-11 16:23:23,202] Trial 0 finished with value: 0.6678669805741629 and parameters: {'max_features': 3592, 'ngram_min': 1, 'ngram_max': 3}. Best is trial 0 with value: 0.6678669805741629.
[I 2025-11-11 16:23:23,736] Trial 1 finished with value: 0.5451519057036222 and parameters: {'max_features': 4635, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 0 with value: 0.6678669805741629.
[I 2025-11-11 16:23:24,121] Trial 2 finished with value: 0.5490473585263653 and parameters: {'max_features': 1237, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 0 with value: 0.6678669805741629.
[I 2025-11-11 16:23:24,736] Trial 3 finished with value: 0.5341143213473796 and parameters: {'max_features': 1028, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 0 with value: 0.6678669805741629.
[I 2025-11-11 16:23:25,370] Trial 4 finished with value: 0.659430995946509 and parameters: {'max_features': 2169, 'ngram_min': 1, 'ngram_max': 2}. Best is trial 0 with value: 0.6678669805741629.
[I 2025-11-11 16:23:2

Melhores parâmetros: {'max_features': 4081, 'ngram_min': 1, 'ngram_max': 3}
Melhor F1: 0.6734990715087125


## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [45]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])

    return X_train, X_val, X_test

In [ ]:
model = carregar_word2vec()
X_train_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, val, test, 'stems', model)

(929606, 300)


In [ ]:
'''
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)
'''

# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [47]:
def Naive_Bayes_MN(X_train, y_train, X_val, y_val, X_test, alphas):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 

In [48]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes_MN(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))
print("\ncom oversampling")
count_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_count_vec, y_train_oversampling, X_val_count_vec,y_val , X_test_count_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_oversampling_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_oversampling_predicted_NB))

----COUNT VECTORIZER ----
Melhor alpha encontrado: 1.0 (F1 val = 0.8018)
              precision    recall  f1-score   support

           1       0.72      0.78      0.75       417
           0       0.85      0.85      0.85      1273
          -1       0.30      0.24      0.27       168

    accuracy                           0.78      1858
   macro avg       0.62      0.62      0.62      1858
weighted avg       0.77      0.78      0.77      1858

[[ 327   82    8]
 [ 108 1080   85]
 [  22  106   40]]


'\nprint("\ncom oversampling")\ncount_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_count_vec, y_train_oversampling, X_val_count_vec,y_val , X_test_count_vec, \n                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) \nlista_y = [str(c) for c in (test[\'classe\'].unique())]\nprint(metrics.classification_report(\ny_test, count_vec_oversampling_predicted_NB, target_names=set(lista_y)))\nprint(metrics.confusion_matrix(y_test, count_vec_oversampling_predicted_NB))'

In [33]:
print ("----TF-IDF ----")
tfidf_vec_predicted_NB= Naive_Bayes_MN(X_train_tfidf_vec, y_train, X_val_tfidf_vec,y_val , X_test_tfidf_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NB))
print("\n com oversampling")

tfidf_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_tfidf_vec,y_val , X_test_tfidf_vec, 
                                    alphas=[0.01, 0.1, 0.5, 1.0, 3.0,5.0, 6.0, 7.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_oversampling_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_oversampling_predicted_NB))

----TF-IDF ----
Melhor alpha encontrado: 0.01 (F1 val = 0.5645)
              precision    recall  f1-score   support

           1       0.25      0.07      0.11       417
           0       0.69      0.92      0.79      1273
          -1       0.14      0.03      0.05       168

    accuracy                           0.65      1858
   macro avg       0.36      0.34      0.32      1858
weighted avg       0.54      0.65      0.57      1858

[[  13  402    2]
 [  32 1240    1]
 [   9  159    0]]

 com oversampling
Melhor alpha encontrado: 7.0 (F1 val = 0.6631)
              precision    recall  f1-score   support

           1       0.51      0.65      0.57       417
           0       0.84      0.61      0.71      1273
          -1       0.19      0.45      0.27       168

    accuracy                           0.61      1858
   macro avg       0.51      0.57      0.52      1858
weighted avg       0.71      0.61      0.64      1858

[[269  91  57]
 [224 782 267]
 [ 37  55  76]]


In [34]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))


[I 2025-11-11 15:08:18,378] A new study created in memory with name: no-name-a36bd325-c3a6-4635-81e9-bfcfb3605e6a
[I 2025-11-11 15:08:18,482] Trial 1 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 2.949569882446841e-07}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,499] Trial 5 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.1263800014039e-05}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,509] Trial 2 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.1213165208214825e-12}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,511] Trial 6 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.609319623025647e-12}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,528] Trial 0 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.2947520782381067e-06}. Best is trial 1 with val

----WORD2VEC ----


[I 2025-11-11 15:08:18,610] Trial 8 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.4834192983414675e-05}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,636] Trial 9 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 8.577984109736401e-12}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,651] Trial 11 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.63028177776144e-10}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,657] Trial 10 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.3349879436850483e-08}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,706] Trial 13 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 7.244245988120601e-12}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-11 15:08:18,719] Trial 14 finished with value: 0.4016961525191454 and param

Melhores hiperparâmetros encontrados:
{'var_smoothing': 2.949569882446841e-07}
              precision    recall  f1-score   support

           1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
          -1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]


In [ ]:
#Glove

## SVM

In [35]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [36]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

[I 2025-11-11 15:08:41,414] A new study created in memory with name: no-name-b383358a-4d3a-48f6-9f6e-1f4d8622cc66


----COUNT VECTORIZER ----


[I 2025-11-11 15:08:42,662] Trial 6 finished with value: 0.271047227926078 and parameters: {'C': 4.348664013511701, 'kernel': 'poly', 'gamma': 'auto', 'degree': 5}. Best is trial 6 with value: 0.271047227926078.
[I 2025-11-11 15:08:43,060] Trial 5 finished with value: 0.2728571162413656 and parameters: {'C': 0.4128041026288293, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,193] Trial 0 finished with value: 0.272029175469006 and parameters: {'C': 0.20074550984619463, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,251] Trial 7 finished with value: 0.26886900665290203 and parameters: {'C': 0.06227384659617249, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 5 with value: 0.2728571162413656.
[I 2025-11-11 15:08:43,403] Trial 2 finished with value: 0.2782224244910812 and parameters: {'C': 2.106516381508683, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best 

Melhores hiperparâmetros encontrados:
{'C': 7.739094578920957, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

           1       0.20      0.12      0.15       417
           0       0.68      0.78      0.73      1273
          -1       0.13      0.11      0.12       168

    accuracy                           0.57      1858
   macro avg       0.34      0.34      0.33      1858
weighted avg       0.52      0.57      0.54      1858

[[ 50 341  26]
 [173 996 104]
 [ 26 123  19]]


In [37]:
print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM= svm(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, tfidf_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_SVM))

[I 2025-11-11 15:08:54,499] A new study created in memory with name: no-name-d4b27d55-029c-47c2-b536-e77953e56d19


---- TF-IDF ----


[I 2025-11-11 15:08:55,173] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 0.012325279805459869, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,293] Trial 0 finished with value: 0.271047227926078 and parameters: {'C': 0.9277300707410308, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,432] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 0.012977477690234215, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,467] Trial 5 finished with value: 0.271047227926078 and parameters: {'C': 0.21656092956323936, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:08:55,671] Trial 7 finished with value: 0.280653324131585 and parameters: {'C': 0.27235873079065787, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 7

Melhores hiperparâmetros encontrados:
{'C': 2.653216629374672, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.23      0.11      0.14       417
           0       0.69      0.88      0.77      1273
          -1       0.11      0.02      0.03       168

    accuracy                           0.63      1858
   macro avg       0.34      0.33      0.32      1858
weighted avg       0.53      0.63      0.56      1858

[[  44  365    8]
 [ 135 1121   17]
 [  15  150    3]]


In [39]:
print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

[I 2025-11-11 15:09:35,410] A new study created in memory with name: no-name-c5f44939-d87b-45d7-b70b-c822e11faef4


----WORD2VEC ----


[I 2025-11-11 15:09:37,628] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 0.3853407159473238, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:37,951] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 0.038993382676786, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:38,323] Trial 6 finished with value: 0.271047227926078 and parameters: {'C': 0.013677459196437055, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.271047227926078.
[I 2025-11-11 15:09:38,518] Trial 5 finished with value: 0.4338411420906391 and parameters: {'C': 0.8597224524711686, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 5 with value: 0.4338411420906391.
[I 2025-11-11 15:09:38,936] Trial 2 finished with value: 0.271047227926078 and parameters: {'C': 1.365233127757608, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 5 

Melhores hiperparâmetros encontrados:
{'C': 9.082446685549941, 'kernel': 'rbf', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.59      0.44      0.51       417
           0       0.76      0.90      0.83      1273
          -1       0.33      0.09      0.14       168

    accuracy                           0.72      1858
   macro avg       0.56      0.48      0.49      1858
weighted avg       0.69      0.72      0.69      1858

[[ 185  225    7]
 [ 104 1146   23]
 [  22  131   15]]


## KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

----COUNT VECTORIZER ----
Best k: 151
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       417
           1       0.68      0.98      0.80      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.23      0.33      0.27      1858
weighted avg       0.47      0.67      0.55      1858

[[   0  417    0]
 [  29 1244    0]
 [   0  168    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
#TF-IDF

#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

#Glove


----COUNT VECTORIZER ----
Best k: 151
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       417
           1       0.68      0.98      0.80      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.23      0.33      0.27      1858
weighted avg       0.47      0.67      0.55      1858

[[   0  417    0]
 [  29 1244    0]
 [   0  168    0]]
----WORD2VEC ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 11
              precision    recall  f1-score   support

          -1       0.62      0.27      0.37       417
           1       0.73      0.95      0.82      1273
           0       0.20      0.01      0.02       168

    accuracy                           0.71      1858
   macro avg       0.51      0.41      0.41      1858
weighted avg       0.65      0.71      0.65      1858

[[ 112  304    1]
 [  56 1210    7]
 [  13  153    2]]


## Decison Tree

In [25]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))


In [ ]:

#TF-IDF

#Word2vec
print ("----COUNT VECTORIZER ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
lista_y = [str(c) for c in (test['classe'].unique())]
print(metrics.classification_report(
y_test, word2vec_predicted_DT, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))


#Glove


----COUNT VECTORIZER ----
Melhor alpha: 0.0037 


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


              precision    recall  f1-score   support

          -1       0.08      0.03      0.04       417
           1       0.67      0.90      0.77      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.62      1858
   macro avg       0.25      0.31      0.27      1858
weighted avg       0.48      0.62      0.54      1858

[[  12  405    0]
 [ 126 1147    0]
 [   4  164    0]]
----COUNT VECTORIZER ----
Melhor alpha: 0.0047 
              precision    recall  f1-score   support

          -1       0.41      0.30      0.35       417
           1       0.72      0.88      0.79      1273
           0       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.38      0.39      0.38      1858
weighted avg       0.59      0.67      0.62      1858

[[ 126  291    0]
 [ 153 1120    0]
 [  26  142    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


## Random Florest

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, make_scorer

def objective(X_train, y_train, X_val, y_val, trial):
    n_estimators = trial.suggest_categorical('n_estimators', [50, 100, 200, 300])
    max_depth = trial.suggest_categorical('max_depth', [5, 10, 15, 20])
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
        random_state=42
    )

    clf.fit(X_train, y_train)

    predictions_rf = clf.predict(X_val)

    f1 = f1_score(y_val, predictions_rf, pos_label=1,average='macro')
    
    return f1

def random_florest(X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial),
                   n_trials=50, n_jobs=-1)

    print("Melhores hiperparâmetros Random Forest:")
    print(study.best_params)

    best_rf = RandomForestClassifier(**study.best_params, random_state=42)
    best_rf.fit(X_train, y_train)

    predictions_rf = best_rf.predict(X_test)
    
    return predictions_rf


## Neural network

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (100, 50)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        learning_rate_init=learning_rate_init,
        max_iter=300,
        random_state=42
    )

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

def neural_network (X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)
    print("Melhores parâmetros:", study.best_params)

    best_params = study.best_params
    best_model = MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=300,
        random_state=42
    )

    best_model.fit(X_train, y_train)
    previsoes = best_model.predict(X_test)
    
    return previsoes


## Regressão Logística

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    # Hiperparâmetros a otimizar
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])  # compatíveis com L1 e L2
    max_iter = trial.suggest_int('max_iter', 100, 1000)

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=max_iter,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    f1 = f1_score(y_val, preds, average='macro')
    return f1


def logistic_regression(X_train, y_train, X_val, y_val, X_test, y_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)

    best_params = study.best_params
    best_model = LogisticRegression(
        **best_params,
        random_state=42
    )

    best_model.fit(X_train, y_train)

    previsoes = best_model.predict(X_test)

    return previsoes
